In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Fouille de données

Mounted at /content/drive
/content/drive/MyDrive/Fouille de données


In [2]:
import sys
import os
import site
sys.path.append(os.path.join(os.path.dirname(site.getsitepackages()[0]), "site-packages"))
!pip install scikit-learn
!pip install imblearn
!pip install dask
! python -m pip install scikit-learn-intelex
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 90.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
import os 
import pandas as pd 
import numpy as np
from sklearnex import patch_sklearn
patch_sklearn("SVC")

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import BorderlineSMOTE
import dask.dataframe as dd
import fastparquet
from sklearn.svm import OneClassSVM
from sklearn.feature_selection import VarianceThreshold
from sklearn.neighbors import KNeighborsClassifier

train = pd.read_parquet('train.parquet.gzip')
test = pd.read_parquet('test.parquet.gzip')

Xtrain = train.drop(["FlagImpaye", "ZIBZIN", "Date", "Heure_split", "DateTransaction", "CodeDecision", "Unnamed: 0"], axis = 1)

Ytrain = pd.DataFrame(train.FlagImpaye)
Ytrain = train['FlagImpaye'].astype('int')

Xtest  = test.drop(["FlagImpaye","ZIBZIN", "Date", "Heure_split", "DateTransaction","CodeDecision", 'Unnamed: 0'], axis = 1)
Ytest  = test.FlagImpaye

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
from xgboost import XGBRFClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import average_precision_score, confusion_matrix
from sklearn.preprocessing import Normalizer

# Application des algorithmes 

def applyAlgo(algo, Xtrain, Ytrain, Xtest, Ytest):

    # On commence par indiquer ce que l'on va faire avec chaque algorithme.
    # on prendra soin de préciser les hyper-paramètres dont dépend l'algorithme

    algo = SVC()
    algo.fit(Xtrain, Ytrain)
    rankTrain = algo.predict(Xtrain)
    rankTest = algo.predict(Xtest)

   # Cette deuxième partie permet d'indiquer quelle est la mesure de performance que 
   # vous souhaitez considérer pour votre étude en cours
    ctest = confusion_matrix(Ytest, rankTest)
    ftest = round(2*ctest[1,1]/(2*ctest[1,1]+ctest[0,1]+ctest[1,0]),4)

    return (ftest)

def Echantillon(number, Xtrain, Ytrain):
    Xtrain = Xtrain.sample(n = number)
    Ytrain = Ytrain.sample(n = number)

    return number, Xtrain, Ytrain

def Processing(method, Xtrain, Ytrain, Xtest):

    if method == "NoProcessing":
        pass
    
    if method == "SelectFromModel": 
        clf = ExtraTreesClassifier(n_estimators=50)
        clf = clf.fit(Xtrain, Ytrain)

        model = SelectFromModel(clf, prefit=True)

        Xtrain = Xtrain[Xtrain.columns[model.get_support(indices = True)]]
        Xtest = Xtest[Xtest.columns[model.get_support(indices = True)]]

    if method == "Variance": 
        selector = VarianceThreshold(threshold = 0.8)
        selector.fit_transform(Xtrain)

        Xtrain = Xtrain[Xtrain.columns[selector.get_support(indices = True)]]
        Xtest = Xtest[Xtest.columns[selector.get_support(indices = True)]]

    return method, Xtrain, Xtest

def SMOTE_S(sampling, Xtrain, Ytrain):

    sm = BorderlineSMOTE(sampling_strategy = sampling,random_state = 0)

    Xtrain, Ytrain = sm.fit_resample(Xtrain, Ytrain)
    
    return sampling, Xtrain, Ytrain


In [ ]:
models = ['SVC']
number = [100000]  #380000 , 999999]
method = ['NoProcessing', 'Variance']
sampling = [0, 0.3 , 0.5]

import time
Fmesure = []
execution = []
algo = []
n_data = []
variable_selector = []
smote = []

columns = ["Classifieur", "Echantillon", "Selection Variable", "Performance", "Sampling", "Temps_execution"] # rajouter pipeline ici si on veut comparer les pré processing

for s in sampling :
    for x in number :
        for l in method : 
            for model in models : 

                if s == 0:
                  x, X_sample_train, Y_sample_train = Echantillon(x, Xtrain, Ytrain)
                  n_data.append(x)
                else:
                  s, X_sample_train, Y_sample_train = SMOTE_S(s, Xtrain, Ytrain)
                  smote.append(s)

                  x, X_sample_train, Y_sample_train = Echantillon(x, X_sample_train, Y_sample_train)
                  n_data.append(x)


                l, X_sample_train, X_sample_test = Processing(l, X_sample_train, Y_sample_train, Xtest)
                variable_selector.append(l)

                # Stocker l'algorithme qui tourne
                algo.append(model)
                start = time.time()
                # Fin normalisation
                apTest = applyAlgo(model, X_sample_train, Y_sample_train, X_sample_test, Ytest)
                Fmesure.append(apTest) 
                end = time.time()
                elapsed = end - start 
                execution.append(elapsed)




In [ ]:
columns = ["Classifieur", "Echantillon", "Selection Variable", "Performance", "Temps_execution"] # rajouter pipeline ici si on veut comparer les pré processing


In [ ]:
score_df = pd.DataFrame(list(zip(algo, n_data, variable_selector, Fmesure, smote, execution)), columns= columns) 
score_df.to_csv("resultats_SVM.csv")